# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

In [ ]:
!pip install scikit-learn==1.1.3

## Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor 
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_validate
import warnings

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/datasets/autos.csv')

In [ ]:
data.info()

In [ ]:
data.head()

Приводим наименования столбцов к строчным буквам

In [ ]:
columns_new = []
for col in data.columns:
    columns_new.append(re.sub('(?<!^)(?=[A-Z])', '_', col ).lower())
data.columns = columns_new

In [ ]:
data.columns

Оцениваем пропущенные значения

In [ ]:
data.isna().sum()

In [ ]:
data_columns = ['vehicle_type','gearbox','model','fuel_type','repaired']

for col in data_columns:
    print('В столбце {} нулевых значений {:.1%}'. format(col, data[col].isna().value_counts()[1] / len(data)))

In [ ]:
data_copy = data.copy()

Найдем и удалим дубликаты

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates().reset_index(drop=True)

Исследуем значения в столбцах на предмет артефактов.

In [ ]:
for col in data.columns:
    print(data[col].unique())

Заполним данные по vehicle_type - тип кузова, fuel_type - тип топлива, gearbox - тип коробки передач наиболее встречающимися по группе моделей автомобиля

In [ ]:
def zero_fill (col):
    data[col].value_counts()[0]
    data[col].replace(np.nan,inplace = True)   
zero_fill('model')

In [ ]:
data['vehicle_type'] = data['vehicle_type'].fillna(data.groupby('model')['vehicle_type']
                                                       .transform(lambda x: x.value_counts().idxmax()))
data['gearbox'] = data['gearbox'].fillna(data.groupby('model')['gearbox']
                                             .transform(lambda x: x.value_counts().idxmax()))
data['fuel_type'] = data['fuel_type'].fillna(data.groupby('model')['fuel_type']
                                                 .transform(lambda x: x.value_counts().idxmax()))

Обработаем целевую переменную.

In [ ]:
#4.price
data['price'].hist(bins=50, grid = True)
plt.title("Распределение стоимости автомобилей")
plt.xlabel('Стоимость автомобилей', fontsize=10)
plt.show()

In [ ]:
data.boxplot(column='price')
plt.title('Разброс значений стоимоcти автомобилей',fontsize=20, loc= 'center')
plt.show()

In [ ]:
data['price'].quantile([0.05,0.95])

Отбросим выбросы целевой переменной и сформируем данные с учетом стоимости автомобиля от 250 евро до 14800 евро

Мощность автомобиля отфильтруем в интервале от 16 до 1600 л.с.

In [ ]:
data[data['power']<400]['power'].hist(bins=100, grid = True)

In [ ]:
data = data[(data['power']>38) & (data['power']<2028)]

В данных о дате регистрации автомобиля много встречается записей несоответствующих году. Максимальный год регистрации автомобиля выберем по дате скачивания анкеты - 2016, минимальный - начало 20 века 

In [ ]:
data['date_crawled'].max()

In [ ]:
data = data[(data['registration_year']>=1900) & (data['registration_year']<=2016)]

Учитывая, что машины как правило чаще ремонтируются чем не ремонтируются, пропуски данных заполним "yes".

In [ ]:
data['repaired'] = data['repaired'].fillna('yes')

In [ ]:
#превратила "repaired" в категориальный бинарный признак
data.loc[data['repaired'] == 'yes','repaired'] = '1'
data.loc[data['repaired'] == 'no','repaired'] = '0'
data = data.astype({'repaired':'int64'})

In [ ]:
data.duplicated().sum()

In [ ]:
data.shape

In [ ]:
data.isna().sum()

Исключим данные не влияющие на цену авто

DateCrawled — дата скачивания анкеты из базы      
DateCreated — дата создания анкеты   
NumberOfPictures — количество фотографий автомобиля   
PostalCode — почтовый индекс владельца анкеты (пользователя)  
LastSeen — дата последней активности пользователя  
RegistrationMonth — месяц регистрации автомобиля  

In [ ]:
def plot_cat (col, price):
    data.plot(x=col,y=price, kind='scatter')
    plt.title(f'Зависимость цены машины от {col}')
    plt.xlabel(f'{col}, ед.', fontsize=10)
    plt.show()

In [ ]:
plot_cat('number_of_pictures', 'price')

In [ ]:
plot_cat('registration_month', 'price')

In [ ]:
plot_cat('postal_code', 'price')

На графиках видно, что выбраные признаки, не связанные напрямую с характеристиками машины и информацией о ее эксплуатации, не влияют на цену автомобиля.

In [ ]:
data_new = data.drop(['date_crawled', 'date_created', 'number_of_pictures',
       'postal_code', 'last_seen', 'registration_month'],axis = 1)

In [ ]:
data.shape[0]/data_copy.shape[0]

**Вывод**
1. Загружены и исследованы данные
2. Удалены аномальные значения, выбросы, лишние признаки
3. Для целей обучения и предсказания осталось 77 процентов от первоначальной выборки

## Обучение моделей

Готовим выходные данные по метрикам качества в разрезе моделей 

In [ ]:
table_of_rmse=[]
table_of_time_ob=[]
table_of_time_pr=[]
table_of_model=[]

Готовим данные для LinearRegression

In [ ]:
#категориальные признаки для OHE LR
object_cols_cat = [col for col in data_new.columns if data_new[col].dtype == 'object']

In [ ]:
object_cols_cat

In [ ]:
#Разделяем обработанный датафрейм на обучающую и тестовую выборки
features = data_new.drop('price', axis=1)
target = data_new['price']
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=42)

In [ ]:
#категориальные признаки для OHE LR
ohe_features_LR = features_train.select_dtypes(include='object').columns.to_list()
print(ohe_features_LR)

In [ ]:
#категориальные признаки для OHE RandomForestRegressor
ohe_features_rfr = ohe_features_LR.copy()
ohe_features_rfr.remove('model')
ohe_features_rfr

In [ ]:
#численные признаки
#'repaired' — категориальный бинарный признак.
num_features = features_train.select_dtypes(exclude='object').columns.to_list()
num_features.remove('repaired')
num_features

In [ ]:
models_train = set(features_train['model'].unique())
models_test = set(features_test['model'].unique())
num_models_train = len(models_train)
num_models_test = len(models_test)
print(f'''
Количество уникальных значений признака "model" 
в обеих выборках одинаковое: {num_models_train == num_models_test}
''')
print(f'''
Уникальные значения признака "model" 
в обеих выборках одинаковые: {models_train == models_test}
''')
print(f'''
Только в тренировочной выборке есть значения: {models_train - models_test}
''')
print(f'''
Только в тестовой выборке есть значения: {models_test - models_train}
''')

In [ ]:
print(features_train.shape,features_test.shape,target_train.shape,target_test.shape)

Кодируем данные для линейной регрессии

In [ ]:
features_train_ohe = features_train.copy()
features_test_ohe = features_test.copy()

In [ ]:
ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

In [ ]:
ohe_col_train = pd.DataFrame(ohe.fit_transform(features_train_ohe[ohe_features_LR]))
ohe_col_test = pd.DataFrame(ohe.transform(features_test_ohe[ohe_features_LR]))

In [ ]:
ohe_col_train.index = features_train_ohe.index
ohe_col_test.index = features_test_ohe.index

In [ ]:
X_train_cat = features_train_ohe.drop(ohe_features_LR,axis =1)
X_test_cat = features_test_ohe.drop(ohe_features_LR,axis =1)

In [ ]:
X_train_ohe = pd.concat([X_train_cat,ohe_col_train],axis =1)
X_test_ohe = pd.concat([X_test_cat,ohe_col_test],axis =1)

In [ ]:
X_train_ohe.head()

In [ ]:
X_test_ohe.head()

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train_ohe[num_features] = scaler.fit_transform(X_train_ohe[num_features])
X_test_ohe[num_features] = scaler.transform(X_test_ohe[num_features])

In [ ]:
X_train_ohe.head()

In [ ]:
X_train_ohe.shape

In [ ]:
X_test_ohe.shape

**LinearRegression**

In [ ]:
model_LR = LinearRegression()

In [ ]:
RMSE_LR = (cross_val_score(model_LR, 
                           X_train_ohe, 
                           target_train, 
                           cv=5, 
                           scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('RMSE LinearRegression =', RMSE_LR)

Результат получился не плохой, вот что значит правильно закодировать данные!!!! Даже вписался в требуемое качество.

In [ ]:
#расчет времени обучения
start_time = time.time()
model_LR.fit(X_train_ohe,target_train)
end_time = time.time()
fit_time_LR = end_time - start_time

In [ ]:
#расчет времени предсказания
start_time = time.time()
pred = model_LR.predict(X_train_ohe) 
end_time = time.time()
pred_time_LR = end_time - start_time

In [ ]:
#формируем данные для оценки
table_of_model.append ('model_LR')
table_of_rmse.append(RMSE_LR)
table_of_time_ob.append (fit_time_LR)
table_of_time_pr.append (pred_time_LR)

Готовим данные для нелинейных моделей

In [ ]:
#Разделяем обработанный датафрейм на обучающую и тестовую выборки
features = data_new.drop('price', axis=1)
target = data_new['price']
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Кодируем и стандартизируем данные для DecisionTreeRegressor и RandomForestRegressor

In [ ]:
ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)
ohe_col_train = pd.DataFrame(ohe.fit_transform(features_train[ohe_features_rfr]))
ohe_col_test = pd.DataFrame(ohe.transform(features_test[ohe_features_rfr]))
ohe_col_train.index = features_train.index
ohe_col_test.index = features_test.index
X_train_cat = features_train.drop(ohe_features_rfr,axis =1)
X_test_cat = features_test.drop(ohe_features_rfr,axis =1)
X_train_ohe = pd.concat([X_train_cat,ohe_col_train],axis =1)
X_test_ohe = pd.concat([X_test_cat,ohe_col_test],axis =1)

In [ ]:
X_train_ohe.head()

In [ ]:
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
enc_train = pd.DataFrame(enc.fit_transform(X_train_ohe[['model']]))
enc_test = pd.DataFrame(enc.transform(X_test_ohe[['model']]))
enc_train.index = X_train_ohe.index
enc_test.index = X_test_ohe.index
X_train = X_train_ohe.drop('model',axis =1)
X_test = X_test_ohe.drop('model',axis =1)
features_train_enc = pd.concat([X_train,enc_train],axis =1)
features_test_enc = pd.concat([X_test,enc_test],axis =1)

In [ ]:
features_train_enc.head()

In [ ]:
scaler = StandardScaler()
features_train_enc[num_features] = scaler.fit_transform(features_train_enc[num_features])
features_test_enc[num_features] = scaler.transform(features_test_enc[num_features])

In [ ]:
features_train_enc.head()

**DecisionTreeRegressor**

In [ ]:
param = {
    'max_depth':[1,2,4,6], 
    'min_samples_split':[18,20,22,24], 
    'min_samples_leaf':[3,4,5,6,7,8] 
}
model = DecisionTreeRegressor()
seach = RandomizedSearchCV(model, param, cv=5, random_state=12345)
seach.fit(features_train_enc, target_train) 

In [ ]:
print('Лучшие параметры модели: ',seach.best_params_)

In [ ]:
model_DTR= DecisionTreeRegressor(min_samples_split = seach.best_params_['min_samples_split'], 
                                 min_samples_leaf = seach.best_params_['min_samples_leaf'], 
                                 max_depth = seach.best_params_['min_samples_leaf'])
RMSE_DTR = (cross_val_score(model_DTR, 
                           features_train_enc, 
                           target_train,
                           cv=5, 
                           scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('RMSE DecisionTreeRegressor =', RMSE_DTR)

In [ ]:
start_time = time.time()
model_DTR.fit(features_train_enc,target_train)
end_time = time.time()
fit_time_DTR = end_time - start_time

In [ ]:
start_time = time.time()
pred = model_DTR.predict(features_train_enc) 
end_time = time.time()
pred_time_DTR = end_time - start_time

In [ ]:
#записываем очередные результаты в итоговую таблицу
table_of_rmse.append(round(RMSE_DTR,2))
table_of_time_ob.append (fit_time_DTR)
table_of_time_pr.append (pred_time_DTR)
table_of_model.append ('model_DTR') 

**RandomForestRegressor**

In [ ]:
param = {
    'n_estimators':[20,30],
    'max_features':[0.3,1]
}
model = RandomForestRegressor()
seach = GridSearchCV(model, param, cv=5)
seach.fit(features_train_enc, target_train) 
print('Лучшие параметры модели: ', seach.best_params_)

In [ ]:
model_RFR= RandomForestRegressor(max_features= seach.best_params_['max_features'],
                                 n_estimators= seach.best_params_['n_estimators'])

In [ ]:
RMSE_RFR = (cross_val_score(model_RFR, 
                           features_train_enc, 
                           target_train,
                           cv=5, 
                           scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('RMSE RandomForestRegressor =', RMSE_RFR)

In [ ]:
start_time = time.time()
model_RFR.fit(features_train_enc,target_train)
end_time = time.time()
fit_time_RFR = end_time - start_time

In [ ]:
start_time = time.time()
pred = model_RFR.predict(features_train_enc) 
end_time = time.time()
pred_time_RFR = end_time - start_time

In [ ]:
#записываем результаты в таблицу
table_of_rmse.append(round(RMSE_RFR,2))
table_of_time_ob.append (fit_time_RFR)
table_of_time_pr.append (pred_time_RFR)
table_of_model.append ('model_RFR') 

Готовим данные для бустинга

In [ ]:
for i in features_train.columns:
    if features_train.loc[:,i].dtype == 'object':
        features_train.loc[:,i] = features_train.loc[:,i].astype('category')

In [ ]:
for i in features_test.columns:
    if features_test.loc[:,i].dtype == 'object':
        features_test.loc[:,i] = features_test.loc[:,i].astype('category')

**LGBMRegressor**

In [ ]:
param_grid = {'learning_rate': [0.1, 0.3, 0.7],
              'max_depth': [15, 20],
             }
model = LGBMRegressor(n_jobs = 3, categorical_feature = object_cols_cat)
search = GridSearchCV(estimator = model, 
                           param_grid = param_grid, 
                           cv = 3,
                           n_jobs = -1, 
                           verbose = 0, 
                           scoring = 'neg_mean_squared_error'
                          )
search.fit(features_train,target_train)

In [ ]:
print(search.best_params_)

In [ ]:
model_LGBMR = LGBMRegressor(learning_rate = search.best_params_['learning_rate'], 
                            max_depth = search.best_params_['max_depth'], random_state=12345)

In [ ]:
RMSE_LGBMR = (cross_val_score(model_LGBMR, 
                           features_train, 
                           target_train,
                           cv=5, 
                           scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('RMSE LGBMRegressor =', RMSE_LGBMR)

In [ ]:
start_time = time.time()
model_LGBMR.fit(features_train,target_train)
end_time = time.time()
fit_time_LGBMR = end_time - start_time

In [ ]:
start_time = time.time()
pred = model_LGBMR.predict(features_train)
end_time = time.time()
pred_time_LGBMR = end_time - start_time

In [ ]:
table_of_rmse.append(round(RMSE_LGBMR,2))
table_of_time_ob.append (fit_time_LGBMR)
table_of_time_pr.append (pred_time_LGBMR)
table_of_model.append ('model_LGBMR') 

**Catboost**

In [ ]:
parameter = {'learning_rate': [0.03,0.1],
             'depth': [4,6,8],
             'l2_leaf_reg': [1,2,3]
              }
model = CatBoostRegressor(cat_features=object_cols_cat)
search = GridSearchCV(estimator = model, param_grid = parameter, cv = 4, n_jobs=-1, scoring='neg_mean_squared_error')
search.fit(features_train, target_train)
print(search.best_params_)

In [ ]:
print(search.best_params_)

In [ ]:
model_CBR = CatBoostRegressor(cat_features=object_cols_cat, depth =search.best_params_['depth'],
                             l2_leaf_reg = search.best_params_['l2_leaf_reg'],
                             learning_rate = search.best_params_['learning_rate'])

In [ ]:
RMSE_CBR = (cross_val_score(model_CBR, 
                           features_train, 
                           target_train,
                           cv=5, 
                           scoring='neg_mean_squared_error').mean() * -1) ** 0.5
print('RMSE CatBoostRegressor =', RMSE_CBR)

In [ ]:
start_time = time.time()
model_CBR.fit(features_train,target_train)
end_time = time.time()
fit_time_CBR = end_time - start_time

In [ ]:
start_time = time.time()
pred = model_CBR.predict(features_train)
end_time = time.time()
pred_time_CBR = end_time - start_time

In [ ]:
table_of_rmse.append(round(RMSE_CBR,2))
table_of_time_ob.append (fit_time_CBR)
table_of_time_pr.append (pred_time_CBR)
table_of_model.append ('model_CBR') 

## Анализ моделей

**Исследовано 5 моделей по 3 параметрам**   
- качество предсказания
- скорость предсказания
- время обучения<br>
Анализ скорости и времени предсказания представлен в таблице

In [ ]:
models = (pd.DataFrame(
    {'Model':table_of_model,'RMSE':table_of_rmse, 'time_study':table_of_time_ob, 'time_predict':table_of_time_pr})
                .sort_values(by='RMSE')
                .set_index('Model'))

models

Значение метрики RMSE, в соответствии с заданием, должно быть меньше 2500. Все модели соответствуют заданному условию. Лучшей моделью по метрике качества стал градиентный бустинг Catboost с результатом RMSE = 1311 с самым большим временем обучения.

По времени обучения и предсказания лучшие показатели у модели DecisionTreeRegressor.

Выполним предсказания на лучшей моделе по метрике качества RMSE.

In [ ]:
start_time = time.time()
pred = model_CBR.predict(features_test)
end_time = time.time()
pred_time_good = end_time - start_time

In [ ]:
RMSE_final_ = mean_squared_error(target_test, pred) ** 0.5
print('RMSE на тестовой выборке - ', RMSE_final_)
print('Время предсказания на лучшей модели на тестовой выборке - ',pred_time_good)

**Значение метрики RMSE на тестовой выборке, в соответствии с заданием, должно быть меньше 2500**

**Вывод**
1. По результатам анализа данных были заполнены пропуски, исключены лишние признаки, аномальные значения и выбросы
2. Выполнено сравнение моделей с использованием различных наборов гиперпараметров.
3. Выбрана лучшая модель по результатам метрики RMSE и времени обучения.<br>
Лучшей моделью по RMSE стал Catboost с результатом в 1311.21. При этом скорость обучения и время предсказания не самые быстрые и занимают последние места среди рассмотренных моделей. Самым быстрым оказался DecisionTreeRegressor на подготовленных данных. Однако у этой модели не самый лучший результат по качеству - 2094.36.
4. Для использовании на практике рекомендуется модель с лучшей метрикой качества - Catboost.